# Matching Registry and PA State Business License Data

In [1]:
import pandas as pd
import mwdsbe
import mwdsbe.datasets.licenses as licenses
import schuylkill as skool
import time

In [2]:
def drop_duplicates_by_date(df, date_column):
    df.sort_values(by=date_column, ascending=False, inplace=True)
    df = df.loc[~df.index.duplicated(keep="first")]
    df.sort_index(inplace=True)
    return df

## Data

In [3]:
registry = mwdsbe.load_registry() # geopandas df
license = licenses.CommercialActivityLicenses().get()

In [4]:
registry.head()

,company_name,dba_name,owner_first,owner_last,location,location_city,location_state,zip_code,mailing_address,mailing_city,mailing_state,mailing_zip,certification_type,capability,local,out_of_state,location_standard,lat,lng,geometry
registry_id,,,,,,,,,,,,,,,,,,,,
0,119 Degrees Architects,NaN,Rafael,Utrera,"1503 Green Street, Suite # 4",Philadelphia,PA,19130.0,"1503 Green Street, Suite # 4",Philadelphia,PA,19130.0,MBE,"NAICS 5413 Architectural, Engineering, and Rel...",True,False,1503 GREEN ST,39.964275,-75.163042,POINT (-75.16304 39.96427)
1,12Bravo Group,NaN,JEFFREY,YEKENCHIK,236 McKendimen Road,Medford Lakes,NJ,8055.0,236 McKendimen Road,Medford Lakes,NJ,8055.0,DSBE,"Addition, alteration and renovation for-sale b...",False,True,NaN,NaN,NaN,None
2,1st Choice Financial Group,ProVisio,Kathrina,Nease,133 N. 21st Street,Camp Hill,PA,17011.0,133 N. 21st Street,Camp Hill,PA,17011.0,WBE,NAICS 928120 International Affairs,False,False,NaN,NaN,NaN,None
3,212 Harakawa Inc.,Two Twelve,Ann,Harakawa,"236 W 27th Street, Suite 802",New York,NY,10001.0,"236 W 27th Street, Suite 802",New York,NY,10001.0,MWBE,Graphic Design Services; Graphic design servic...,False,True,NaN,NaN,NaN,None
4,215 Media Solutions,NaN,Dewain,Johnson,810 Felton Avenue,Sharon Hill,PA,19079.0,810 Felton Avenue,Sharon Hill,PA,19079.0,MBE,NAICS 5414 Specialized Design Services ; NAICS...,False,False,NaN,NaN,NaN,None


In [5]:
state_license = pd.read_csv('./data/PAStateBusinessLicense/Sales_Tax_Licenses_and_Certificates_Current_Monthly_County_Revenue.csv')

In [6]:
print('Size of state_license data:', len(state_license))

Size of state_license data: 347532


In [7]:
# convert state_license column names from titlecase to snakecase
def to_snake_case(aList):
    res = []
    for item in aList:
        words = item.strip().lower().split(' ')
        item = '_'.join(words)
        res.append(item)
    return res

In [8]:
state_license.columns = to_snake_case(state_license.columns.tolist())

In [9]:
# clean data
ignore_words = ['inc', 'group', 'llc', 'corp', 'pc', 'incorporated', 'ltd', 'co', 'associates', 'services', 'company', 'enterprises', 'enterprise', 'service', 'corporation']
cleaned_registry = skool.clean_strings(registry, ['company_name', 'dba_name'], True, ignore_words)
cleaned_license = skool.clean_strings(license, ['company_name'], True, ignore_words)
cleaned_state_license = skool.clean_strings(state_license, ['legal_name', 'trade_name'], True, ignore_words)

cleaned_registry = cleaned_registry.dropna(subset=['company_name'])
cleaned_license = cleaned_license.dropna(subset=['company_name'])
cleaned_state_license = cleaned_state_license.dropna(subset=['legal_name'])

In [10]:
len(cleaned_license)

203541

In [11]:
cleaned_state_license.head()

,county,legal_name,trade_name,postal_code,country,account_number,license_type,expiration_date,address_with_lat/long
0,ADAMS,nugen energy,nugen energy,17331,US,67**9551,Retail,09/30/2021,"1601 CARLISLE PIKE\nHANOVER, PA 17331\n(39.834..."
1,ALLEGHENY,cellux,mobilexpress west mifflin,15123,US,85**6245,Retail,01/31/2021,"3075 CLAIRON RD CENTURY III MA\nWEST MIFFLIN, ..."
2,ALLEGHENY,dina persichetti hill,nails at last,15101,US,81**6740,Retail,02/29/2024,"4085 WM FLYNN HWY STE 10A\nALLISON PARK, PA 15101"
3,ADAMS,elwood mummert,mummert elwood k,17331-7700,US,81**7441,Retail,01/31/2021,"700 PINE GROVE RD\nHANOVER, PA 17331-7700\n(39..."
4,ALLEGHENY,cosmograce,brillobox,15224,US,67**9185,Retail,09/30/2021,"4104 PENN AVE\nPITTSBURGH, PA 15224\n(40.46574..."


In [12]:
# just getting PA state in registry
pa_registry = cleaned_registry[cleaned_registry.location_state == 'PA']

In [13]:
len(pa_registry)

1769

## Merge registry and state_license by company_name and legal_name / trade name

In [96]:
t1 = time.time()
merged = (
    skool.tf_idf_merge(pa_registry, cleaned_state_license, left_on="company_name", right_on="legal_name", score_cutoff=85)
    .pipe(skool.tf_idf_merge, pa_registry, cleaned_state_license, left_on="company_name", right_on="trade_name", score_cutoff=85)
    .pipe(skool.tf_idf_merge, pa_registry, cleaned_state_license, left_on="dba_name", right_on="legal_name", score_cutoff=85)
    .pipe(skool.tf_idf_merge, pa_registry, cleaned_state_license, left_on="dba_name", right_on="trade_name", score_cutoff=85)
)
t = time.time() - t1

In [97]:
print('Execution time:', t/60, 'min')

Execution time: 21.944276861349742 min


In [100]:
matched = merged.dropna(subset=['legal_name', 'dba_name'], how='all')

In [101]:
len(matched)

585

In [102]:
matched_state = matched

In [56]:
# matched.to_excel (r'C:\Users\dabinlee\Desktop\mwdsbe\data\state_license\pa-registry-full-state-license\tf-idf-85.xlsx', header=True)

In [14]:
matched_state = pd.read_excel(r'C:\Users\dabinlee\Desktop\mwdsbe\data\state_license\pa-registry-full-state-license\tf-idf-85.xlsx')

In [103]:
len(matched_state)

585

In [104]:
exact_matches = matched_state[matched_state.match_probability == 1]

In [105]:
len(exact_matches)

139

##### Eliminate companies with different zip code

In [106]:
matched_state['postal_code_clean'] = matched_state.postal_code.astype(str).apply(lambda x : x.split("-")[0]).astype(float)

C:\Users\dabinlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [107]:
matched_state = matched_state.set_index('left_index')

KeyError: "None of ['left_index'] are in the columns"

In [115]:
len(matched_state)

585

In [109]:
# matched_state_zip = matched_state[matched_state.zip_code == matched_state.postal_code_clean]

In [110]:
# len(matched_state_zip)

247

In [116]:
matched_state['expiration_date'] = pd.to_datetime(matched_state['expiration_date'], errors='coerce')

C:\Users\dabinlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [117]:
matched_state = drop_duplicates_by_date(matched_state, 'expiration_date')

C:\Users\dabinlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dabinlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [118]:
len(matched_state) # state_license, same zip code, without duplicates

564

## Comparing between match between "registry-opendata license" and "registry-state license"
How many more new companies do we get from registry-opendata license matching?

In [26]:
# t1 = time.time()
# merged = (
#     skool.tf_idf_merge(cleaned_registry, cleaned_license, on="company_name", score_cutoff=85)
#     .pipe(skool.tf_idf_merge, cleaned_registry, cleaned_license, left_on="dba_name", right_on="company_name", score_cutoff=85)
# )
# t = time.time() - t1

In [27]:
# print('Execution time:', t/60, 'min')

In [28]:
# matched_openphilly_license = merged.dropna(subset=['company_name_y'])

In [29]:
# len(matched_openphilly_license)

In [30]:
# matched_openphilly_license.issue_date = matched_openphilly_license.issue_date.astype(str)

In [31]:
# matched_openphilly_license.to_excel (r'C:\Users\dabinlee\Desktop\mwdsbe\data\license-opendataphilly\tf-idf\tf-idf-85.xlsx', header=True)

##### Loading matched of registry and opendataphilly_license data

In [120]:
matched_opendataphilly_license = pd.read_excel(r'C:\Users\dabinlee\Desktop\mwdsbe\data\license-opendataphilly\tf-idf\tf-idf-85.xlsx')

In [121]:
matched_opendataphilly_license = matched_opendataphilly_license.set_index('left_index')

In [122]:
len(matched_opendataphilly_license)

1642

In [123]:
matched_opendataphilly_license = drop_duplicates_by_date(matched_opendataphilly_license, "issue_date") # without duplicates

C:\Users\dabinlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [124]:
len(matched_opendataphilly_license)

1502

In [125]:
matched_opendataphilly_license.tail()

,company_name_x,dba_name,owner_first,owner_last,location,location_city,location_state,zip_code,mailing_address,mailing_city,...,location_standard,lat,lng,geometry,right_index,match_probability,license_num,issue_date,license_status,company_name_y
left_index,,,,,,,,,,,,,,,,,,,,,
3107,zavorski masonry restoration,NaN,Lisa,Zavorski,717 Ford Avenue,Langhorne,PA,19047.0,717 Ford Avenue,Langhorne,...,NaN,NaN,NaN,NaN,101652,1.000000,343160,2005-10-31 00:00:00+00:00,Active,zavorski masonry restoration
3110,zenga engineering,NaN,Gwendolyn,Lodise,313 East Broad Street,Palmyra,NJ,8065.0,313 East Broad Street,Palmyra,...,NaN,NaN,NaN,NaN,136814,1.000000,509641,2010-07-26 00:00:00+00:00,Active,zenga engineering
3113,zizza highway,NaN,Arlene,Zizza,382 CONCHESTER HIGHWAY,Glen Mills,PA,19342.0,382 CONCHESTER HIGHWAY,Glen Mills,...,NaN,NaN,NaN,NaN,189864,1.000000,770177,2018-03-23 00:00:00+00:00,Active,zizza highway
3115,zones,NaN,Mr.,Lalji,1102 15TH ST SW,Auburn,WA,98001.0,1102 15TH ST SW,Auburn,...,NaN,NaN,NaN,NaN,26525,0.860842,145187,2002-05-06 00:00:00+00:00,Active,zone
3117,zora,NaN,Charles,O'Reilly,1901 Dorcas Lane,Wilmington,DE,19806.0,1901 Dorcas Lane,Wilmington,...,NaN,NaN,NaN,NaN,88625,1.000000,33338,2005-01-28 00:00:00+00:00,Active,zora


In [126]:
# unique company?
len(matched_opendataphilly_license.index.unique()) # yes

1502

In [127]:
diff = matched_state.index.difference(matched_opendataphilly_license.index).tolist()

In [128]:
len(diff)

201

In [129]:
matched_state.loc[diff][['company_name', 'dba_name', 'legal_name', 'trade_name']]

,company_name,dba_name,legal_name,trade_name
2,1st choice financial,provisio,provisions,provisions
14,4x3,NaN,4x3,4x3
28,a n s steel,NaN,a n s steel,ans steel
65,acclaim systems,NaN,acclaim systems,acclaim systems
72,ace construction maintenance more,ace construction maintenance more,NaN,NaN
...,...,...,...,...
3054,westside framing studio,westside studio,NaN,NaN
3059,whaumbush dba detention physicians,detention physicians,NaN,NaN
3068,wilson500,NaN,wilson500,wilson500
3099,your quality solutions,NaN,your quality solutions,your quality solutions


In [130]:
# newly matched: matching with state_license data
difference = matched_state.loc[diff]

In [131]:
difference_SL = difference

In [361]:
# difference.to_excel (r'C:\Users\dabinlee\Desktop\mwdsbe\data\difference.xlsx', header=True)

## Investigate missing companies in opendataphilly license data
We found 94 newly matched companies from matching between registry and state_license, why these are not appeared in matching between registry and opendataphilly license data?

In [44]:
t1 = time.time()
merged = (
    skool.tf_idf_merge(cleaned_registry, cleaned_license, on="company_name", score_cutoff=0)
    .pipe(skool.tf_idf_merge, cleaned_registry, cleaned_license, left_on="dba_name", right_on="company_name", score_cutoff=0)
)
t = time.time() - t1

In [45]:
print('Execution time:', t/60, 'min')

Execution time: 3.9544533332188925 min


In [90]:
matched = merged.dropna(subset=['company_name_y'])

In [91]:
len(matched)

3165

In [49]:
matched = drop_duplicates_by_date(matched, 'issue_date')

C:\Users\dabinlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dabinlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [52]:
# matched.issue_date = matched.issue_date.astype(str)
# matched.to_excel(r'C:\Users\dabinlee\Desktop\mwdsbe\data\tf-idf0.xlsx', header=True)

In [50]:
matched

,company_name_x,dba_name,owner_first,owner_last,location,location_city,location_state,zip_code,mailing_address,mailing_city,...,location_standard,lat,lng,geometry,right_index,match_probability,license_num,issue_date,license_status,company_name_y
0,119 degrees architects,NaN,Rafael,Utrera,"1503 Green Street, Suite # 4",Philadelphia,PA,19130.0,"1503 Green Street, Suite # 4",Philadelphia,...,1503 GREEN ST,39.964275,-75.163042,POINT (-75.16304 39.96427),131105.0,1.000000,480115.0,2009-07-31 00:00:00+00:00,Active,119 degrees architects
1,12bravo,NaN,JEFFREY,YEKENCHIK,236 McKendimen Road,Medford Lakes,NJ,8055.0,236 McKendimen Road,Medford Lakes,...,NaN,NaN,NaN,None,108854.0,0.566374,379054.0,2006-07-26 00:00:00+00:00,Active,bravo
2,1st choice financial,provisio,Kathrina,Nease,133 N. 21st Street,Camp Hill,PA,17011.0,133 N. 21st Street,Camp Hill,...,NaN,NaN,NaN,None,37296.0,0.742458,174332.0,2004-10-12 00:00:00+00:00,Active,omni choice financial
3,212 harakawa,two twelve,Ann,Harakawa,"236 W 27th Street, Suite 802",New York,NY,10001.0,"236 W 27th Street, Suite 802",New York,...,NaN,NaN,NaN,None,4417.0,0.489846,82446.0,1995-10-06 00:00:00+00:00,Active,baraka
4,215 media solutions,NaN,Dewain,Johnson,810 Felton Avenue,Sharon Hill,PA,19079.0,810 Felton Avenue,Sharon Hill,...,NaN,NaN,NaN,None,147975.0,0.668217,565674.0,2012-05-30 00:00:00+00:00,Active,forte media solutions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114,zoeza,NaN,Jason,Wingard,"536 W. Moreland Avenue, Suite C.",Philadelphia,PA,19118.0,"536 W. Moreland Avenue, Suite C.",Philadelphia,...,536 W MORELAND AVE,40.059613,-75.207085,POINT (-75.20708 40.05961),122356.0,0.789721,436094.0,2008-02-20 00:00:00+00:00,Active,the zoeza
3115,zones,NaN,Mr.,Lalji,1102 15TH ST SW,Auburn,WA,98001.0,1102 15TH ST SW,Auburn,...,NaN,NaN,NaN,None,26525.0,0.860842,145187.0,2002-05-06 00:00:00+00:00,Active,zone
3116,zook motors,NaN,Susan,Wilson,25 Willow Run Drive,Kane,PA,16735.0,25 Willow Run Drive,Kane,...,NaN,NaN,NaN,None,136273.0,0.553686,505624.0,2010-06-15 00:00:00+00:00,Active,beaverbrook motor
3117,zora,NaN,Charles,O'Reilly,1901 Dorcas Lane,Wilmington,DE,19806.0,1901 Dorcas Lane,Wilmington,...,NaN,NaN,NaN,None,88625.0,1.000000,33338.0,2005-01-28 00:00:00+00:00,Active,zora


In [94]:
cleaned_registry.loc[cleaned_registry.index.difference(matched.index)]

,company_name,dba_name,owner_first,owner_last,location,location_city,location_state,zip_code,mailing_address,mailing_city,mailing_state,mailing_zip,certification_type,capability,local,out_of_state,location_standard,lat,lng,geometry
5,22,NaN,Caroline,Harper,757 Public Road,Bethlehem,PA,18015.0,757 Public Road,Bethlehem,PA,18015.0,WBE,All Other Specialty Trade Contractors ; Enviro...,False,False,NaN,NaN,NaN,None
14,4x3,NaN,Amy,Siano,"63 W Lancaster Ave, Ste 4",Ardmore,PA,19003.0,"63 W Lancaster Ave, Ste 4",Ardmore,PA,19003.0,WBE,Graphic Design Services; Graphic design servic...,False,False,NaN,NaN,NaN,None
181,ama resource,NaN,Anne Marie,Auld,P.O. Box 9796,Wilmington,DE,19809.0,P.O. Box 9796,Wilmington,DE,19809.0,WBE,Waste materials merchant wholesalers ; Materi...,False,True,NaN,NaN,NaN,None
197,american renewable energy,NaN,Wei,Wang,"255 Old New Brunswick Road, Suite N280",Piscataway,NJ,8854.0,"255 Old New Brunswick Road, Suite N280",Piscataway,NJ,8854.0,MBE,26060\tINSULATION;,False,True,NaN,NaN,NaN,None
203,ampm north america,NaN,James,Moody,612 Vernon Road,Philadelphia,PA,19119.0,612 Vernon Road,Philadelphia,PA,19119.0,MBE,26046\tGENERAL CONSTRUCTION-REPAIR; 90611\tHea...,True,False,612 VERNON RD,40.061407,-75.177179,POINT (-75.17718 40.06141)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015,vj of suffolk,NaN,Vijay,Desai,"1090 King Georges Post Road, Suite 301",Edison,NJ,8837.0,"1090 King Georges Post Road, Suite 301",Edison,NJ,8837.0,MBE,Administrative Management and General Manageme...,False,True,NaN,NaN,NaN,None
3017,vkg,NaN,Kimberly,Hawthorne,734 E. Dorset Street,Philadelphia,PA,19119.0,734 E. Dorset Street,Philadelphia,PA,19119.0,MWBE,Computer Training ; Computer Training ; Prof...,True,False,734 DORSET ST,40.062765,-75.175337,POINT (-75.17534 40.06276)
3079,woods peacock engineering consultants,NaN,Carrie,Seifert,"5250 CHEROKEE AVENUE, SUITE 420",Alexandria,VA,22312.0,"5250 CHEROKEE AVENUE, SUITE 420",Alexandria,VA,22312.0,WBE,Engineering consulting services; Engineering s...,False,True,NaN,NaN,NaN,None
3087,wu,NaN,Raymond,Wu,"100 Gaither Drive, Suite C",Mount Laurel,NJ,8054.0,"100 Gaither Drive, Suite C, Suite C",Mount Laurel,NJ,8054.0,MBE,"Addition, alteration and renovation operative ...",False,True,NaN,NaN,NaN,None


In [95]:
cleaned_license.loc[cleaned_license.company_name == '22']

,license_num,issue_date,license_status,company_name
168732,667661.0,2015-06-10 00:00:00+00:00,Active,22


In [59]:
missed_by_OL = matched.loc[difference.index]

C:\Users\dabinlee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [61]:
missed_by_OL

,company_name_x,dba_name,owner_first,owner_last,location,location_city,location_state,zip_code,mailing_address,mailing_city,...,location_standard,lat,lng,geometry,right_index,match_probability,license_num,issue_date,license_status,company_name_y
left_index,,,,,,,,,,,,,,,,,,,,,
28,a n s steel,NaN,Linda,Manning,4340 Sepviva Street,Philadelphia,PA,19124.0,4340 Sepviva Street,Philadelphia,...,4340 SEPVIVA ST,40.003004,-75.079992,POINT (-75.07999 40.00300),56931.0,0.629722,182098.0,2005-01-28 00:00:00+00:00,Inactive,steel
65,acclaim systems,NaN,Kailash,Kalantri,110 EAST PENNSYLVANIA BOULEVARD,Feasterville,PA,19053.0,110 EAST PENNSYLVANIA BOULEVARD,Feasterville,...,NaN,NaN,NaN,None,31063.0,0.568845,164123.0,2003-07-16 00:00:00+00:00,Active,l m systems
100,advanced technology and systems integrators,NaN,Venkata,Gundala,209 Heather Road,Upper Darby,PA,19082.0,209 Heather Road,Upper Darby,...,NaN,NaN,NaN,None,130738.0,0.795560,478125.0,2009-07-01 00:00:00+00:00,Active,advanced technology and syst int
115,affairs to be remembered,NaN,Eileen,Williamson-Getty,555 Abbott Dr,Broomall,PA,19008.0,555 Abbott Dr,Broomall,...,NaN,NaN,NaN,None,50976.0,0.469441,32121.0,2005-01-28 00:00:00+00:00,Active,catered affairs
195,american personnel managers and consultants,american personnel managers consultants,Patricia,Gingrich,"3607 ROSEMONT AVENUE, SUITE 101",Camp Hill,PA,17011.0,"3607 ROSEMONT AVENUE, SUITE 101",Camp Hill,...,NaN,NaN,NaN,None,109303.0,0.836791,381151.0,2006-08-14 00:00:00+00:00,Active,american personnel managers an
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3013,visual sound,NaN,MARJORIE,BOGOSIAN,485 PARK WAY,Broomall,PA,19008.0,485 PARK WAY,Broomall,...,NaN,NaN,NaN,None,16797.0,0.697870,118778.0,1999-07-15 00:00:00+00:00,Active,visual solutions
3039,watts window cleaning janitorial,watts facility solutions,Yvette,Watts,"4548 Market Street, M4",Philadelphia,PA,19139.0,"4548 Market Street, M4",Philadelphia,...,4548 MARKET ST,39.958043,-75.212655,POINT (-75.21266 39.95804),135282.0,0.816941,102365.0,2010-04-20 00:00:00+00:00,Active,watts window cleaning
3068,wilson500,NaN,Corinna,Wilson,500 Arlington Road,Camp Hill,PA,17011.0,500 Arlington Road,Camp Hill,...,NaN,NaN,NaN,None,113155.0,0.503507,11690.0,2007-01-25 00:00:00+00:00,Active,wilsons


In [548]:
# missed_by_OL.issue_date = missed_by_OL.issue_date.astype(str)
# missed_by_OL.to_excel (r'C:\Users\dabinlee\Desktop\mwdsbe\data\missing94.xlsx', header=True)

##### median

In [60]:
missed_by_OL.match_probability.median()

0.6752304834235887

In [62]:
difference.match_probability.median()

1.0

##### mean

In [63]:
missed_by_OL.match_probability.mean()

0.6641255379006749

In [65]:
difference.match_probability.mean()

0.991423969309069

##### max & min

In [67]:
missed_by_OL.match_probability.max()

0.8497287372705913

In [68]:
missed_by_OL.match_probability.min()

0.39815105014993596

In [69]:
difference.match_probability.max()

1.0

In [70]:
difference.match_probability.min()

0.858655159823412

In [75]:
difference_OL_SL = difference.merge(missed_by_OL, left_index=True, right_index=True)

In [85]:
difference_SL_OL = difference_OL_SL[['company_name','dba_name_x', 'match_probability_x', 'legal_name', 'trade_name', 'match_probability_y', 'company_name_y']]

In [87]:
difference_SL_OL.to_excel(r'C:\Users\dabinlee\Desktop\mwdsbe\data\94-difference-SL-OL.xlsx', header=True)

In [86]:
difference_SL_OL

,company_name,dba_name_x,match_probability_x,legal_name,trade_name,match_probability_y,company_name_y
left_index,,,,,,,
28,a n s steel,NaN,1.000000,a n s steel,ans steel,0.629722,steel
65,acclaim systems,NaN,1.000000,acclaim systems,acclaim systems,0.568845,l m systems
100,advanced technology and systems integrators,NaN,0.859795,advanced technology systems,advanced technology systems integ,0.795560,advanced technology and syst int
115,affairs to be remembered,NaN,1.000000,affairs to be remembered,affairs to be rememberd,0.469441,catered affairs
195,american personnel managers and consultants,american personnel managers consultants,0.888080,american personnel managers,american personnel managers consu,0.836791,american personnel managers an
...,...,...,...,...,...,...,...
3013,visual sound,NaN,1.000000,visual sound,visual sound,0.697870,visual solutions
3039,watts window cleaning janitorial,watts facility solutions,1.000000,watts window cleaning janitorial,watts window cleaning janitorial,0.816941,watts window cleaning
3068,wilson500,NaN,1.000000,wilson500,wilson500,0.503507,wilsons


### Compare intersection between matched_opendataphilly_license and matched_state_zip
* matched_opendataphilly_license: matched data between pa_registry and license data from opendataphilly
* matched_state_zip: matched data between pa_registry and license data from state_registry and filter matches which do not match zipcodes

In [512]:
intersection = matched_state_zip.index.intersection(matched_opendataphilly_license.index).tolist()

In [513]:
len(intersection) # 246 - 94

152

In [514]:
intersection1 = matched_opendataphilly_license.loc[intersection]

In [515]:
len(intersection1)

152

In [516]:
intersection2 = matched_state_zip.loc[intersection]

In [517]:
len(intersection2)

152

In [518]:
intersection1 = intersection1[['company_name_x', 'dba_name', 'match_probability', 'company_name_y']]

In [519]:
intersection2 = intersection2[['match_probability', 'legal_name', 'trade_name']]

In [520]:
intersection = intersection1.merge(intersection2, left_index=True, right_index=True)

In [521]:
intersection

,company_name_x,dba_name,match_probability_x,company_name_y,match_probability_y,legal_name,trade_name
left_index,,,,,,,
16,6 degrees consulting,NaN,1.000000,6 degrees consulting,1.0,6 degrees consulting,6 degrees consulting
43,aaaa office warehouse surplus,corporate furniture solutions,0.938296,aaaa office and warehouse surplus,1.0,aaaa office warehouse surplus,aaaa office warehouse surplus
48,abbadon,superior facility,1.000000,abbadon,1.0,abbadon,abbadon
64,accessibility modification solutions,NaN,1.000000,accessibility modification solutions,1.0,accessibility modification solutions,accessibility modification solution
69,accu fire fabrication,NaN,1.000000,accu fire fabrication,1.0,accu fire fabrication,accu fire fabrication
...,...,...,...,...,...,...,...
3022,voith mactavish architects llp,NaN,0.933612,voith mactavish architects,1.0,voith mactavish architects llp,voith mactavish architects llp
3037,wash cycle laundry,wash cycle laundry,1.000000,wash cycle laundry,1.0,wash cycle laundry,wash cycle laundry
3044,webco construction,NaN,1.000000,webco construction,1.0,webco construction,webco construction


In [355]:
# intersection.to_excel (r'C:\Users\dabinlee\Desktop\mwdsbe\data\intersection.xlsx', header=True)

## Merge by address - Not matching well

In [211]:
cleaned_state_license.head()

,county,legal_name,trade_name,postal_code,country,account_number,license_type,expiration_date,address_with_lat/long
0,ADAMS,nugen energy,nugen energy,17331,US,67**9551,Retail,09/30/2021,"1601 CARLISLE PIKE\nHANOVER, PA 17331\n(39.834..."
1,ALLEGHENY,cellux,mobilexpress west mifflin,15123,US,85**6245,Retail,01/31/2021,"3075 CLAIRON RD CENTURY III MA\nWEST MIFFLIN, ..."
2,ALLEGHENY,dina persichetti hill,nails at last,15101,US,81**6740,Retail,02/29/2024,"4085 WM FLYNN HWY STE 10A\nALLISON PARK, PA 15101"
3,ADAMS,elwood mummert,mummert elwood k,17331-7700,US,81**7441,Retail,01/31/2021,"700 PINE GROVE RD\nHANOVER, PA 17331-7700\n(39..."
4,ALLEGHENY,cosmograce,brillobox,15224,US,67**9185,Retail,09/30/2021,"4104 PENN AVE\nPITTSBURGH, PA 15224\n(40.46574..."


In [217]:
# split street address from address_with_lat/long
cleaned_state_license['street_address'] = cleaned_state_license['address_with_lat/long'].astype(str).apply(lambda x : x.split("\n")[0])

In [218]:
cleaned_state_license.head()

,county,legal_name,trade_name,postal_code,country,account_number,license_type,expiration_date,address_with_lat/long,street_address
0,ADAMS,nugen energy,nugen energy,17331,US,67**9551,Retail,09/30/2021,"1601 CARLISLE PIKE\nHANOVER, PA 17331\n(39.834...",1601 CARLISLE PIKE
1,ALLEGHENY,cellux,mobilexpress west mifflin,15123,US,85**6245,Retail,01/31/2021,"3075 CLAIRON RD CENTURY III MA\nWEST MIFFLIN, ...",3075 CLAIRON RD CENTURY III MA
2,ALLEGHENY,dina persichetti hill,nails at last,15101,US,81**6740,Retail,02/29/2024,"4085 WM FLYNN HWY STE 10A\nALLISON PARK, PA 15101",4085 WM FLYNN HWY STE 10A
3,ADAMS,elwood mummert,mummert elwood k,17331-7700,US,81**7441,Retail,01/31/2021,"700 PINE GROVE RD\nHANOVER, PA 17331-7700\n(39...",700 PINE GROVE RD
4,ALLEGHENY,cosmograce,brillobox,15224,US,67**9185,Retail,09/30/2021,"4104 PENN AVE\nPITTSBURGH, PA 15224\n(40.46574...",4104 PENN AVE


In [219]:
pa_registry.head()

,company_name,dba_name,owner_first,owner_last,location,location_city,location_state,zip_code,mailing_address,mailing_city,mailing_state,mailing_zip,certification_type,capability,local,out_of_state,location_standard,lat,lng,geometry
registry_id,,,,,,,,,,,,,,,,,,,,
0,119 degrees architects,NaN,Rafael,Utrera,"1503 Green Street, Suite # 4",Philadelphia,PA,19130.0,"1503 Green Street, Suite # 4",Philadelphia,PA,19130.0,MBE,"NAICS 5413 Architectural, Engineering, and Rel...",True,False,1503 GREEN ST,39.964275,-75.163042,POINT (-75.16304 39.96427)
2,1st choice financial,provisio,Kathrina,Nease,133 N. 21st Street,Camp Hill,PA,17011.0,133 N. 21st Street,Camp Hill,PA,17011.0,WBE,NAICS 928120 International Affairs,False,False,NaN,NaN,NaN,None
4,215 media solutions,NaN,Dewain,Johnson,810 Felton Avenue,Sharon Hill,PA,19079.0,810 Felton Avenue,Sharon Hill,PA,19079.0,MBE,NAICS 5414 Specialized Design Services ; NAICS...,False,False,NaN,NaN,NaN,None
5,22,NaN,Caroline,Harper,757 Public Road,Bethlehem,PA,18015.0,757 Public Road,Bethlehem,PA,18015.0,WBE,All Other Specialty Trade Contractors ; Enviro...,False,False,NaN,NaN,NaN,None
7,24 hour cleaning,NaN,Mary Colleen,Zoltowski,14005 Barcalow Street,Philadelphia,PA,19116.0,14005 Barcalow Street,Philadelphia,PA,19116.0,WBE,Janitorial equipment and supplies merchant who...,True,False,14005 BARCALOW ST,40.131349,-75.014284,POINT (-75.01428 40.13135)


In [220]:
# clean street information
cleaned_pa_registry = skool.clean_strings(pa_registry, ['location'], True)
cleaned_state_license = skool.clean_strings(cleaned_state_license, ['street_address'], True)

cleaned_pa_registry = cleaned_pa_registry.dropna(subset=['location'])
cleaned_state_license = cleaned_state_license.dropna(subset=['street_address'])

In [241]:
t1 = time.time()
merged_by_street = skool.tf_idf_merge(cleaned_pa_registry, cleaned_state_license, left_on='location', right_on='street_address', score_cutoff=95)
t = time.time() - t1

In [242]:
print('Execution time:', t/60, 'min')

Execution time: 7.25786319176356 min


In [243]:
matched_by_street = merged_by_street.dropna(subset=['street_address'])

In [244]:
len(matched_by_street)

179

In [248]:
len(matched_by_street.index.unique()) # bug in tf-idf merge: not doing best match

155

In [249]:
matched_by_street[['company_name', 'location', 'match_probability', 'legal_name', 'trade_name', 'street_address']]

,company_name,location,match_probability,legal_name,trade_name,street_address
17,84 lumber,1019 route 519,1.000000,2100 penn ave,2100 penn ave,1019 route 519
17,84 lumber,1019 route 519,1.000000,84 lumber,84 lumber 84 fitness center,1019 route 519
25,a bob s towing,2220 orthodox street,0.959385,a bobs towing,a bobs towing,2220 orthodox st
28,a n s steel,4340 sepviva street,0.968335,a n s steel,ans steel,4340 sepviva st
33,a lee cook hauling,643 pennfield drive,0.951261,j j specialty,j j specialty,643 pennfield dr
...,...,...,...,...,...,...
3044,webco construction,2512 s 76th street,1.000000,webco construction,webco construction,2512 s 76th street
3055,wf design,4590 main street,1.000000,james morrissey,somo sophi,4590 main street
3062,wilco mechanical,po box 477,1.000000,dew drops,dew drops,po box 477
3062,wilco mechanical,po box 477,1.000000,fredericktown united methodist church,fredericktown united methodist chur,po box 477


In [250]:
# matched_by_street.to_excel (r'C:\Users\dabinlee\Desktop\mwdsbe\data\state_license\by_street\tf-idf-95.xlsx', header=True)

In [251]:
diff = matched_by_street.index.difference(matched_opendataphilly_license.index) # newly catched matches

In [252]:
len(diff)

75

In [257]:
newly_matched_by_street = matched_by_street.loc[diff][['company_name', 'dba_name', 'location', 'legal_name', 'trade_name', 'street_address']]

In [258]:
# newly_matched_by_street.to_excel (r'C:\Users\dabinlee\Desktop\mwdsbe\data\state_license\by_street\tf-idf-95-diff.xlsx', header=True)